<a href="https://colab.research.google.com/github/sjkearney1228/Senior_Design/blob/main/continual_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing PyTorch 0.4.0

Tensorflow is installed by default in Google Colab (guess why :P). If you want to use another DL toolkit you have to do it by yourself. Run the command below to install it. It should take less than a couple of minutes.

Ok, torch is installed and imported! Let' see if it can see the GPU:

In [ ]:
import torch
torch.cuda.is_available()

True

That's great, let us import then a few libraries, which we'll be using during this tutorial!

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

# 15 Activity Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
np.random.seed(1)
X_data = []

In [ ]:
import pathlib
dataset_path=pathlib.Path('/content/drive/MyDrive/Continual_Learning/15_class_dataset/')

In [ ]:
mypath = dataset_path
dirlist = sorted(os.listdir(mypath))
files = sorted(glob.glob ('/content/drive/MyDrive/Continual_Learning/15_class_dataset/*/*.png'))
for myFile in files:
    # print(myFile)
    image = cv2.imread (myFile)
    image = cv2.resize(image, (128,128), interpolation = cv2.INTER_AREA)
    X_data.append (image)

print('X_data shape:', np.array(X_data).shape)

# Create pattern to find files for labeling
dir_arr=['/' + s + '/' for s in dirlist]

# label according to the folder name
labels = []
for i in files:
    res1 = [ele for ele in dir_arr if(ele in i)]
    res  = res1[0][1:-1]


    if res =='01_Vaccuming floor':
        labels.append(0)
    elif res =='02_Normal Walk':
        labels.append(1)
    elif res =='03_short step walk':
        labels.append(2)
    elif res =='04_Walking with a Cane':
        labels.append(3)
    elif res =='05_Dragging furniture':
        labels.append(4)
    elif res =='06_Walk_crutches':
        labels.append(5)
    elif res =='07_Skipping':
        labels.append(6)
    elif res =='08_Marching':
        labels.append(7)
    elif res =='09_Limping':
        labels.append(8)
    elif res =='10_Weight on both hands':
        labels.append(9)
    elif res =='11_weight on one hand':
        labels.append(10)
    elif res =='12_Walking on Toes':
        labels.append(11)
    elif res =='13_Scissor gait walking':
        labels.append(12)
    elif res =='14_books on bookshelf':
        labels.append(13)
    elif res =='15_Folding laundry':
        labels.append(14)

print('Labels shape:', np.array(labels).shape)

X_data shape: (1425, 128, 128, 3)
Labels shape: (1425,)


In [ ]:
X=np.array(X_data)
np_labels=np.array(labels)

In [ ]:
x_train, x_test, t_train, t_test = train_test_split(X, np_labels, test_size=0.20, random_state=42)

x_train = x_train.astype(np.float32)
x_train /= 255.

x_test = x_test.astype(np.float32)
x_test /= 255.

x_train = np.swapaxes(x_train, 1, 3)
# x_train = x_train[:,1,:,:]
# x_train = np.expand_dims(x_train, axis=1)

x_test = np.swapaxes(x_test, 1, 3)
# x_test = x_test[:,1,:,:]
# x_test = np.expand_dims(x_test, axis=1)

# import keras
# t_train = keras.utils.to_categorical(t_train)
# t_test = keras.utils.to_categorical(t_test)

print("x_train dim and type: ", x_train.shape, x_train.dtype)
print("t_train dim and type: ", t_train.shape, t_train.dtype)
print("x_test dim and type: ", x_test.shape, x_test.dtype)
print("t_test dim and type: ", t_test.shape, t_test.dtype)

x_train dim and type:  (1140, 3, 128, 128) float32
t_train dim and type:  (1140,) int64
x_test dim and type:  (285, 3, 128, 128) float32
t_test dim and type:  (285,) int64


# Deep Learning

In [ ]:
# switch to False to use CPU
use_cuda = True

use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu");
torch.manual_seed(1);

... and define our first conv-net! We will use 3 layers of convolutions and two fully connected layers:

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # self.conv2 = nn.Conv2d(10, 30, kernel_size=5)
        self.conv1 = nn.Conv2d(3, 15, kernel_size=5)
        self.conv2 = nn.Conv2d(15, 30, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        # self.fc1 = nn.Linear(320, 50)
        self.fc1 = nn.Linear(25230, 90)
        # self.fc2 = nn.Linear(50, 10)
        self.fc2 = nn.Linear(90, 15)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        print(x.shape)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        print(x.shape)
        # x = x.view(-1, 320)
        x = x.view(-1, 25230)
        print(x.shape)
        x = F.relu(self.fc1(x))
        # print(x.shape)
        x = F.dropout(x, training=self.training)
        # print(x.shape)
        x = self.fc2(x)
        # print(x.shape)
        return x

Then we can write the *train* and *test* functions. Note that for simplicity here we are not using PyTorch [Data Loaders](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) but this is not recommended for efficiency.

In [ ]:
def train(model, device, x_train, t_train, optimizer, epoch):
    model.train()
    
    for start in range(0, len(t_train)-1, 256):
      end = start + 256
      x, y = torch.from_numpy(x_train[start:end]), torch.from_numpy(t_train[start:end]).long()
      x, y = x.to(device), y.to(device)
      
      optimizer.zero_grad()

      print(x.shape)
      output = model(x)
      print(output.shape)
      # print('here cmodel()')
      loss = F.cross_entropy(output, y)
      print('here cross entropy')
      loss.backward()
      optimizer.step()
      #print(loss.item())
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

def test(model, device, x_test, t_test):
    model.eval()
    test_loss = 0
    correct = 0
    for start in range(0, len(t_test)-1, 256):
      end = start + 256
      with torch.no_grad():
        x, y = torch.from_numpy(x_test[start:end]), torch.from_numpy(t_test[start:end]).long()
        x, y = x.to(device), y.to(device)
        output = model(x)
        test_loss += F.cross_entropy(output, y).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= len(t_train)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(t_test),
        100. * correct / len(t_test)))
    return 100. * correct / len(t_test)

Then we are ready to instantiate our model and start the training!

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
for epoch in range(1, 3):
  train(model, device, x_train, t_train, optimizer, epoch)
  test(model, device, x_test, t_test)

Wow! 94% accuracy in such a short time. 

**Questions to explore:**

*   Can you find a better parametrization to improve the final accuracy?
*   Can you change the network architecture to improve the final accuracy?
*   Can you achieve the same performances with a smaller architecture?
*   What's the difference in accuracy if you change convolutions with fully connected layers?

Some tips here: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354